In [1]:
import numpy as np
import simpy
from scipy.stats import expon

## Singel Server

an event has two properties, triggered or processed.
triggered: the event has been added to the pending list, and it is stored in the queue.
processed: the event has happend, and has been removed from the queue.
an event can be:
1. not triggered
2. triggered, not processed
3. triggered and processed

In [2]:
def arrival_p(lmbda):
    while True:
        dt = expon.rvs(scale = 1/ lmbda)
        arrival = env.timeout(dt)
        print("before yield:", arrival.triggered, arrival.processed)
        print("before yield", env.now)
        print("next time:",env.now + dt)
        yield arrival
        print("after yield:", arrival.triggered, arrival.processed)
        print("after yield:", env.now)
env = simpy.Environment()
env.process(arrival_p(lmbda = 1))
env.run(until = 5)

before yield: True False
before yield 0
next time: 0.2033843352370114
after yield: True True
after yield: 0.2033843352370114
before yield: True False
before yield 0.2033843352370114
next time: 0.253042106534575
after yield: True True
after yield: 0.253042106534575
before yield: True False
before yield 0.253042106534575
next time: 2.539957106063712
after yield: True True
after yield: 2.539957106063712
before yield: True False
before yield 2.539957106063712
next time: 3.210252265891008
after yield: True True
after yield: 3.210252265891008
before yield: True False
before yield 3.210252265891008
next time: 4.138280651105686
after yield: True True
after yield: 4.138280651105686
before yield: True False
before yield 4.138280651105686
next time: 4.16955268171543
after yield: True True
after yield: 4.16955268171543
before yield: True False
before yield 4.16955268171543
next time: 5.035686052509937


each arrival starts a seperate service, it starts when arrival happends and ends when service departs system.

resource is when customers might not get the service immediately (if there is a resource there might be a line)

when a customer comes to request the server,rqt = server.request() the following happen:
1. sever is idle, the request will be process and stored in server.users. triggered = True, processed = False
2. not idle, the request will be in server.queue, triggered = False, processed = False

when a reuqest get processed, yield rqt, the following would happen:
1. reuqest already in server.users, the status will simply changed to triggered = True, processed = True 
2. request was in server.queue, we wait until the request moves to server.users, and then triggered = True, processed= True

when a customer leaves, sever.release(rqt), the following will happen:
1. the request will be deleted from server.users
2. trigger and process the rqt corresponding to the next customer
3. now rqt is processed, service() corresponding to the user will resume

We can define a generic event using the code event_name = env.event()

When the event is created, status is triggered= False, processed = False

event.succeed() is called, triggered = True, processed = False, add to the pending waitlist

yield event -> triggered = True, processed = True

In [3]:
import numpy as np
import simpy
from scipy.stats import expon
def arrivals(env,server,S,G,stop):
    while True:
        dt = expon.rvs(scale = 1)
        arrival = env.timeout(dt)
        yield arrival
        if env.now < 10:
            S.append(env.now)
            env.process(service(env,server,S,G,stop))
            # stop processing arrival after 10
        elif len(server.queue) + len(server.users) == 0:
            stop.succeed()
            yield stop

        
        
def service(env,server,S,G,stop):
    rqt = server.request() #you will be added to a list. When you yield the request
    yield rqt #triggered
    service_time = 0.8
    service = env.timeout(service_time)
    yield service
    G.append(env.now)
    server.release(rqt)
        
    
    
            
def simulate():
    S = []
    G = []
    env = simpy.Environment()
    stop  =env.event()
    server = simpy.Resource(env,capacity = 1)
    env.process(arrivals(env,server,S,G,stop))
    env.run(until = stop)
    return max(G[-1] - 10,0)

X = [simulate() for i in range(1000)]
print(np.mean(X))

1.1446720125388805


## Multi-server Queue

### Multi-server queues in a series

Method 1:
After recieve the service, customer can be treated as a new arrival and enter the next service

Method 2:
we can just think of it as a long service

In [4]:
np.random.seed(42)

def arrival(env,servers,S,G):
    i = 0
    while True:
        dt = expon.rvs(scale = 1)
        arrival = env.timeout(dt)
        yield arrival
        S. append(env.now)
        env.process(service(env,servers,S,G,i))
        i = i + 1
        
def service(env,servers,S,G,i):
    n = len(servers)
    for i in range(0,n):
        rqt = servers[i].request()
        yield rqt
        dt = expon.rvs(scale = 1)
        yield env.timeout(dt)
        servers[i].release(rqt)
    G.append(env.now)
    
    
    
    
def simulate(n):
    S = []
    G = []
    env = simpy.Environment()
    servers = [simpy.Resource(env, capacity = 1) for i in range(0,n)]
    env.process(arrival(env,servers,S,G))
    env.run(until = 10)
    return S,G


simulate(3)    

([0.4692680899768591,
  3.4793895208943804,
  3.6489858128089856,
  4.56806796643625,
  4.588867274435389,
  8.0924247495937,
  8.242659278321039,
  8.698936496541886,
  8.921672358670577,
  9.643701513703704,
  9.691265363460111],
 [2.868581207760608,
  5.789146779231948,
  5.991758202064205,
  7.594956214109065,
  8.701187115182845,
  9.944379670765636])

### Multi-server in parapllel with single line

Method: 
1. Create n requests
2. process the requests yield simpy.AnyOf(env,[rqt1,rqt2])
3. determin which server becomes available first  result = yield rqt1 | rqt2 if rqt in result and ...
4. cancel the other requests, delete them from queue  rqt2.cancel(). if both are yielded


In [5]:
np.random.seed(42)

def arrival(env,server,S,G):
    i = 0
    while True:
        dt = expon.rvs(scale = 0.5)
        arrival = env.timeout(dt)
        yield arrival
        S. append(env.now)
        env.process(service(env,server,S,G,i))
        i = i + 1
        
def service(env,server,S,G,i):
    print('customer {} attives at {}'.format(i,env.now))
    rqt = server.request()
    print('there are currently {} customers in service'.format(len(server.users)))
    print('there are currently {} customers in line'.format(len(server.queue)))
    yield rqt
    dt = 1.0
    yield env.timeout(dt)
    print('customer {} leaves at {}'.format(i, env.now))
    G.append(env.now)
    server.release(rqt)
    
    
    
def simulate(j):
    S = []
    G = []
    env = simpy.Environment()
    server = simpy.Resource(env,capacity = j)
    env.process(arrival(env,server,S,G))
    env.run(until = 3)
    return S,G









    
simulate(2)    

customer 0 attives at 0.23463404498842955
there are currently 1 customers in service
there are currently 0 customers in line
customer 0 leaves at 1.2346340449884297
customer 1 attives at 1.7396947604471902
there are currently 1 customers in service
there are currently 0 customers in line
customer 2 attives at 2.398067607219915
there are currently 2 customers in service
there are currently 0 customers in line
customer 1 leaves at 2.73969476044719
customer 3 attives at 2.8545388841078916
there are currently 2 customers in service
there are currently 0 customers in line
customer 4 attives at 2.939351319339065
there are currently 2 customers in service
there are currently 1 customers in line


([0.23463404498842955,
  1.7396947604471902,
  2.398067607219915,
  2.8545388841078916,
  2.939351319339065],
 [1.2346340449884297, 2.73969476044719])

### Multi-server in parallel with multiple lines

customer looking for the shortest line
len(server.queue) + server.count

In [6]:
from scipy.stats import expon, uniform

np.random.seed(42)

def arrival(env,servers, lambdas ,S,G):
    while True:
        dt = expon.rvs(scale = 0.5)
        arrival = env.timeout(dt)
        yield arrival
        S.append(env.now)
        env.process(service(env,servers,lambdas,S,G))
    
def service(env, servers,lambdas, S,G):
    lens = np.array([len(server.queue) + server.count for server in servers])
    ind = np.argmin(lens)
    rqt = servers[ind].request()
    yield rqt
    dt = expon.rvs(scale = 1/lambdas[ind])
    yield env.timeout(dt)
    servers[ind].release(rqt)
    
    G.append(env.now)
    
    
def simulate(n):
    S = []
    G = []
    env = simpy.Environment()
    servers = [simpy.Resource(env,capacity = 1) for i in range(0,n)]
    lambdas = [1,4,3]
    env.process(arrival(env,servers,lambdas,S,G))
    env.run(until=3)
    return S,G


simulate(3)

([0.23463404498842955,
  1.7396947604471902,
  2.196166037335167,
  2.280964183292469],
 [1.5513797385338788, 1.9093196309095364, 2.2560048059438476])

## Advanced Feature

###  Priority Resource

We allow customer to cut line.

Upon arrival, a customer will line up behind people with higher or same priority

line up before people with a low priority

Once the service starts, the service will not be stopped by another person with a higher priority

In [7]:
#priority is in ascending order
#you don't cut somebody currently in service
def arrival():
    i = 0
    while True:
        dt = expon.rvs()
        yield env.timeout(dt)
        #half of the users have priority 0
        if uniform.rvs() < 0.5:
            priority = 0
        else:
            priority = 1
        env.process(service(i,priority))
        i = i + 1
        
def service(i,priority):
    rqt = server.request(priority = priority)
    yield rqt
    dt = expon.rvs()
    yield env.timeout(dt)
    server.release(rqt)
    
env = simpy.Environment()
server = simpy.PriorityResource(env,capacity = 1)
env.process(arrival())
env.run(until = 10)

### interruption

in s2:

s1.interrupt(cause)

in s1:

```
try:
    operations1
except simpy.Interrupt as Inter: 
    if Inter.cause=="needs to sleep":
        operations2
    elif Inter.cause=="need to take a coffee break":
        operations3
    else:
        operation 4
```


In [8]:
import numpy as np
import simpy

from scipy.stats import expon

np.random.seed(42)

def arrivals():
    i = 0
    while True:
        dt = expon.rvs()
        yield env.timeout(dt)
        i = i + 1
        # starts two process, one is the customer trying to access the service, one is call 
        s1 = env.process(service(i))
        env.process(call(i, s1))
        print('customer {} arrives at {}'.format(i, env.now))

def call(i, s1):
    j = 0
    while True:
        if j == 0:
            dt = expon.rvs(scale=1/0.25)
            yield env.timeout(dt)
            call_start = env.now
            j = j + 1
        else:
            dt = expon.rvs(scale=1/0.25)
            if dt < 1.0:
                dt = 1.5
            yield env.timeout(dt)
            call_start = env.now
        if rqt_list[i - 1].processed != True:
            s1.interrupt('customer {} receives call at {}'.format(i, env.now))
        else:
            return
        

def service(i):
    rqt = desk.request()
    rqt_list.append(rqt)
    while True:
        try:
            #if it is not idle, it will be in the queues
            yield rqt
            #wait for the queue to run out
            break
        except simpy.Interrupt:
            print('customer {} Receives call at {}'.format(i, env.now))
            rqt.cancel()#because I know for a fact that if cutomer is processed, it wwould not be interrupted
            yield env.timeout(1.0)
            print('customer {} ends call at {}'.format(i, env.now))
            rqt = desk.request()
            rqt_list[i - 1] = rqt
    # it is yielded. 
    print('customer {} starts service at {}'.format(i, env.now))
    dt = expon.rvs()
    yield env.timeout(dt)
    print('customer {} ends service at {}'.format(i, env.now))
    desk.release(rqt)

env=simpy.Environment()
env.process(arrivals())
rqt_list=[]
call_times=[]
desk=simpy.Resource(env)

T=15
env.run(until=T)

customer 1 arrives at 0.4692680899768591
customer 1 starts service at 0.4692680899768591
customer 1 ends service at 1.3822106437528123
customer 2 arrives at 3.4793895208943804
customer 2 starts service at 3.4793895208943804
customer 2 ends service at 3.539228289503061
customer 3 arrives at 3.649014391356727
customer 3 starts service at 3.649014391356727
customer 3 ends service at 4.880264453061317
customer 4 arrives at 5.660245255836666
customer 4 starts service at 5.660245255836666
customer 5 arrives at 5.681044563835805
customer 6 arrives at 5.919732189084752
customer 7 arrives at 6.122343611917009
customer 5 Receives call at 6.483760518834458
customer 8 arrives at 6.8662714427778635
customer 6 Receives call at 7.370747106926661
customer 4 ends service at 7.446674799191341
customer 7 starts service at 7.446674799191341
customer 8 Receives call at 7.467209557687218
customer 5 ends call at 7.483760518834458
customer 9 arrives at 7.812642316677662
customer 5 Receives call at 7.860652489

### Preemptive Resource

This allows the customer with higher priority being able to kick someone off who is in service and has a lower priority.
```
simpy.PreemptiveResource()
resource.request(priority=a, preempt=True/False)
```

When a customer is kicked, the process corresponding to the server will receive an interruption. The type of this interruption is 
```
simpy.resources.resource.Preempted
```




```
try:
    operations1   
except simpy.Interrupt as Inter: 
    if type(Inter.cause)==simpy.resources.resource.Preempted:#if I got kicked off by some random asshole
         operations2
     else:                 # I probably have to blame myself for some accident
         operations3
```

In [9]:
#what preemptive resource does is it kicks people out of server
np.random.seed(42)

def arrival():
    i = 0
    while True:
        dt = expon.rvs()
        yield env.timeout(dt)
        env.process(service(i))
        i = i + 1
def service(i):
    if uniform.rvs() < 0.25:
        priority = 0
        preempt = True
        print('{} is an emergency victim'.format(i))
    else:
        priority = 2
        preempt = False
        print('{} is a regular patient'.format(i))
    print('{} arrives at {}'.format(i,env.now))
    rqt = server.request(priority = priority, preempt = preempt)
    yield rqt#ok so here is the difference, when yield it is probably in a queue, or being served, until now no one would 
    #interrupt it. 
    print('{} gets the server at {}'.format(i,env.now))
    dt = expon.rvs()
    while True:
        try:
            yield env.timeout(dt)
            print('{} leaves at {}'.format(i,env.now))
            server.release(rqt)
        except:
            print('{} kicked off at {}'.format(i,env.now))
            rqt = server.request(priority = 1)
            yield rqt
            print('{} restarted service at {}'.format(i,env.now))
    server.release()

    

    
env = simpy.Environment()
server = simpy.PreemptiveResource(env,capacity = 1)
env.process(arrival())
env.run(until = 10)

0 is a regular patient
0 arrives at 0.4692680899768591
0 gets the server at 0.4692680899768591
0 leaves at 1.3822106437528123
0 leaves at 2.295153197528766
0 leaves at 3.208095751304719
1 is an emergency victim
1 arrives at 3.4793895208943804
1 gets the server at 3.4793895208943804
1 leaves at 3.539228289503061
1 leaves at 3.599067058111742
2 is a regular patient
2 arrives at 3.649014391356727
2 gets the server at 3.649014391356727
1 leaves at 3.6589058267204226
1 leaves at 3.7187445953291034
1 leaves at 3.778583363937784
1 leaves at 3.838422132546465
1 leaves at 3.8982609011551457
1 leaves at 3.9580996697638264
1 leaves at 4.017938438372507
1 leaves at 4.077777206981188
0 leaves at 4.121038305080672
1 leaves at 4.137615975589869
1 leaves at 4.1974547441985495
1 leaves at 4.25729351280723
1 leaves at 4.317132281415911
1 leaves at 4.376971050024592
1 leaves at 4.4368098186332725
1 leaves at 4.496648587241953
1 leaves at 4.556487355850634
1 leaves at 4.616326124459315
1 leaves at 4.67616